# Import

In [818]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
from tensorflow.keras.layers import Dense, SimpleRNN, Flatten
from keras import Model, Sequential, layers, regularizers, optimizers
from keras.callbacks import EarlyStopping
from overcome_tomorrow.utils.data import *
from overcome_tomorrow.ml_logic.preprocess import *
from overcome_tomorrow.ml_logic.model import *
from tqdm import tqdm
pd.set_option('display.max_columns', None)
from sklearn import set_config; set_config(display='diagram')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load model

In [2]:
garmin_data = merge_all_data("../raw_data/Wellness/", "../raw_data/Fitness/", "../raw_data/Aggregator/")
activities = pd.read_csv("../raw_data/activities.csv", parse_dates=["timestamp", "start_time"])

✅ ../raw_data/Wellness/ loaded
✅ Wellness cleaned
✅ ../raw_data/Fitness/ loaded
✅ Fitness cleaned


/home/nico/code/nicblanc/overcome-tomorrow/overcome_tomorrow/utils/data.py:180: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_fitness_df["date"] = pd.to_datetime(clean_fitness_df["beginTimestamp"], unit="ms").dt.date
/home/nico/code/nicblanc/overcome-tomorrow/overcome_tomorrow/utils/data.py:181: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_fitness_df["date"] = pd.to_datetime(clean_fitness_df["date"])
/home/nico/code/nicblanc/overcome-tomorrow/overcome_tomorrow/utils/data.py:182: SettingWithCopy

✅ ../raw_data/Aggregator/ loaded
✅ Aggregator cleaned
✅ Datasets merged


In [3]:
garmin_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3445 entries, 2019-08-29 to 2024-03-16
Data columns (total 52 columns):
 #   Column                                 Non-Null Count  Dtype         
---  ------                                 --------------  -----         
 0   start_sleep                            3445 non-null   datetime64[ns]
 1   end_sleep                              3445 non-null   datetime64[ns]
 2   deepSleepSeconds                       3440 non-null   float64       
 3   lightSleepSeconds                      3440 non-null   float64       
 4   remSleepSeconds                        3421 non-null   float64       
 5   awakeSleepSeconds                      3440 non-null   float64       
 6   qualityScore                           1902 non-null   float64       
 7   beginTimestamp                         3383 non-null   datetime64[ns]
 8   activityTrainingLoad                   3361 non-null   float64       
 9   activityType                           3383 n

In [4]:
garmin_data.sort_values(by=["beginTimestamp"], inplace=True)
garmin_data.dropna(subset = ["beginTimestamp"], inplace=True)
garmin_data.tail(3)

,start_sleep,end_sleep,deepSleepSeconds,lightSleepSeconds,remSleepSeconds,awakeSleepSeconds,qualityScore,beginTimestamp,activityTrainingLoad,activityType,aerobicTrainingEffect,aerobicTrainingEffectMessage,anaerobicTrainingEffect,anaerobicTrainingEffectMessage,avgBikeCadence,avgHr,avgPower,avgRunCadence,avgSpeed,calories,caloriesConsumed,distance,duration,maxHr,maxPower,maxRunCadence,maxSpeed,moderateIntensityMinutes,normPower,sportType,trainingEffectLabel,trainingStressScore,vigorousIntensityMinutes,totalKilocalories,activeKilocalories,totalDistanceMeters,wellnessDistanceMeters,highlyActiveSeconds,activeSeconds,floorsAscendedInMeters,floorsDescendedInMeters,restingHeartRate,minAvgHeartRate,maxAvgHeartRate,hydration_sweatLossInML,respiration_avgWakingRespirationValue,respiration_highestRespirationValue,respiration_lowestRespirationValue,bodyBattery_Highest,bodyBattery_Lowest,allDayStress_awake,allDayStress_asleep
calendarDate,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2024-03-15,2024-03-14 20:07:00,2024-03-15 02:47:53,2760.0,20460.0,720.0,60.0,73.0,2024-03-15 05:25:09,144.809784,gravel_cycling,3.5,IMPACTING_TEMPO_22,0.0,NO_ANAEROBIC_BENEFIT_0,77.0,127.0,218.0,NaN,0.8657,8459.65038,NaN,7.261684e+06,8.388064e+06,142.0,418.0,NaN,1.1915,1.0,221.0,CYCLING,TEMPO,147.399994,129.0,4204.0,2252.0,82879.0,10263.0,1676.0,4010.0,70.453,48.237,40.0,40.0,177.0,1594.0,13.0,19.0,8.0,78.0,25.0,26.0,17.0
2024-03-15,2024-03-14 20:07:00,2024-03-15 02:47:53,2760.0,20460.0,720.0,60.0,73.0,2024-03-15 10:57:38,0.604935,strength_training,0.0,NO_AEROBIC_BENEFIT_18,0.0,NO_ANAEROBIC_BENEFIT_0,NaN,75.0,NaN,NaN,0.0000,79.61038,NaN,0.000000e+00,4.022590e+05,90.0,NaN,NaN,NaN,NaN,NaN,FITNESS_EQUIPMENT,UNKNOWN,NaN,NaN,4204.0,2252.0,82879.0,10263.0,1676.0,4010.0,70.453,48.237,40.0,40.0,177.0,1594.0,13.0,19.0,8.0,78.0,25.0,26.0,17.0
2024-03-15,2024-03-14 20:07:00,2024-03-15 02:47:53,2760.0,20460.0,720.0,60.0,73.0,2024-03-15 11:14:42,22.125320,running,1.5,RECOVERY_5,0.0,NO_ANAEROBIC_BENEFIT_0,NaN,126.0,319.0,84.0,0.3521,586.60280,NaN,2.337730e+05,6.639260e+05,139.0,442.0,88.0,0.4749,0.0,323.0,RUNNING,RECOVERY,NaN,10.0,4204.0,2252.0,82879.0,10263.0,1676.0,4010.0,70.453,48.237,40.0,40.0,177.0,1594.0,13.0,19.0,8.0,78.0,25.0,26.0,17.0


In [5]:
activities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3363 entries, 0 to 3362
Data columns (total 35 columns):
 #   Column                           Non-Null Count  Dtype              
---  ------                           --------------  -----              
 0   timestamp                        3363 non-null   datetime64[ns, UTC]
 1   total_anaerobic_training_effect  3358 non-null   float64            
 2   enhanced_avg_speed               3361 non-null   float64            
 3   avg_stroke_distance              332 non-null    float64            
 4   total_training_effect            3358 non-null   float64            
 5   num_active_lengths               317 non-null    float64            
 6   max_running_cadence              714 non-null    float64            
 7   max_heart_rate                   3356 non-null   float64            
 8   sub_sport                        3363 non-null   object             
 9   total_descent                    2392 non-null   float64            
 10  

In [6]:
activities.sort_values(by=["start_time"], inplace=True)
activities.reset_index(drop=True, inplace=True)
activities.tail(3)

,timestamp,total_anaerobic_training_effect,enhanced_avg_speed,avg_stroke_distance,total_training_effect,num_active_lengths,max_running_cadence,max_heart_rate,sub_sport,total_descent,start_time,total_distance,pool_length,178,total_calories,max_cadence,sport,188,avg_step_length,enhanced_max_speed,205,num_lengths,206,207,pool_length_unit,normalized_power,training_load_peak,total_ascent,avg_power,total_strokes,training_stress_score,avg_cadence,avg_heart_rate,max_power,activity_id
3360,2024-03-15 07:45:23+00:00,0.0,8.657,NaN,3.5,NaN,NaN,142.0,gravel_cycling,298.0,2024-03-15 05:25:09+00:00,72616.84,NaN,1371.0,2019.0,104.0,cycling,3.0,NaN,11.915,95.0,NaN,67.0,67.0,NaN,221.0,144.809784,296.0,218.0,10737.0,147.4,77.0,127.0,418.0,nicko64@hotmail.fr_241349530713
3361,2024-03-15 11:05:33+00:00,0.0,0.000,NaN,0.0,NaN,NaN,90.0,strength_training,NaN,2024-03-15 10:57:38+00:00,0.00,NaN,39.0,19.0,NaN,training,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.604935,NaN,NaN,NaN,NaN,NaN,75.0,NaN,nicko64@hotmail.fr_241374800667
3362,2024-03-15 11:26:17+00:00,0.0,3.521,NaN,1.5,NaN,88.0,139.0,generic,13.0,2024-03-15 11:14:42+00:00,2337.73,NaN,184.0,140.0,88.0,running,1.0,1265.6,4.749,95.0,NaN,91.0,91.0,NaN,323.0,22.125320,16.0,319.0,NaN,NaN,84.0,126.0,442.0,nicko64@hotmail.fr_241377639543


# Fit Preprocessors

In [508]:
preproc_garmin_data = create_preproc_garmin_data(garmin_data)
preproc_activity = create_preproc_activity(activities)

In [509]:
preproc_garmin_data

ColumnTransformer(transformers=[('knn_transformer',
                                 Pipeline(steps=[('knn_imputer',
                                                  KNNImputer(n_neighbors=4)),
                                                 ('robust_scaler',
                                                  RobustScaler())]),
                                 Index(['deepSleepSeconds', 'lightSleepSeconds', 'remSleepSeconds',
       'awakeSleepSeconds', 'qualityScore', 'activityTrainingLoad',
       'aerobicTrainingEffect', 'anaerobicTrainingEffect', 'avgBikeCadence',
       'avgHr', 'avgPower...
      dtype='object')),
                                ('pipe_categorical',
                                 Pipeline(steps=[('simple_imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('one_hot',
                                                  OneHotEncoder(drop='if_binary',
                                                                handle_unknown='ignore',
                                                                sparse_output=False))]),
                                 ['sportType', 'trainingEffectLabel']),
                                ('cycle transform', CyclicalFeaturesSleep(),
                                 Index(['start_sleep', 'end_sleep', 'beginTimestamp'], dtype='object'))])

In [510]:
preproc_activity

InvertableColumnTransformer(transformers=[('knn_imputer',
                                           Pipeline(steps=[('knn_imputer',
                                                            KNNImputer(n_neighbors=4)),
                                                           ('robust_scaler',
                                                            RobustScaler())]),
                                           ['training_load_peak',
                                            'avg_heart_rate', '206',
                                            'max_heart_rate',
                                            'total_anaerobic_training_effect',
                                            'total_calories', '188',
                                            'total_distance',
                                            'enhanced_avg_speed',
                                            'total_training_effect', '178',
                                            'total_ascent...
                                                            SimpleImputer(fill_value=100,
                                                                          strategy='constant')),
                                                           ('robust_scaler',
                                                            RobustScaler())]),
                                           ['205']),
                                          ('cat_encoder',
                                           Pipeline(steps=[('simple_imputer',
                                                            SimpleImputer(strategy='most_frequent')),
                                                           ('one_hot',
                                                            OneHotEncoder(drop='if_binary',
                                                                          handle_unknown='ignore',
                                                                          sparse_output=False))]),
                                           ['sport']),
                                          ('cycle_encoder',
                                           CyclicalFeaturesActivity(),
                                           ['timestamp', 'start_time'])])

In [42]:
preproc_garmin_data.fit(garmin_data)
preproc_activity.fit(activities)

ColumnTransformer(transformers=[('knn_imputer',
                                 Pipeline(steps=[('knn_imputer',
                                                  KNNImputer(n_neighbors=4)),
                                                 ('robust_scaler',
                                                  RobustScaler())]),
                                 ['training_load_peak', 'avg_heart_rate', '206',
                                  'max_heart_rate',
                                  'total_anaerobic_training_effect',
                                  'total_calories', '188', 'total_distance',
                                  'enhanced_avg_speed', 'total_training_effect',
                                  '178', 'total_ascent', 'enhance...
                                                  SimpleImputer(fill_value=100,
                                                                strategy='constant')),
                                                 ('robust_scaler',
                                                  RobustScaler())]),
                                 ['205']),
                                ('cat_encoder',
                                 Pipeline(steps=[('simple_imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('one_hot',
                                                  OneHotEncoder(drop='if_binary',
                                                                handle_unknown='ignore',
                                                                sparse_output=False))]),
                                 ['sport']),
                                ('cycle_encoder', CyclicalFeaturesActivity(),
                                 ['timestamp', 'start_time'])])

# Create slide windows

In [11]:
preproc_activity.transform(activities.iloc[[50]])

,knn_imputer__training_load_peak,knn_imputer__avg_heart_rate,knn_imputer__206,knn_imputer__max_heart_rate,knn_imputer__total_anaerobic_training_effect,knn_imputer__total_calories,knn_imputer__188,knn_imputer__total_distance,knn_imputer__enhanced_avg_speed,knn_imputer__total_training_effect,knn_imputer__178,knn_imputer__total_ascent,knn_imputer__enhanced_max_speed,knn_imputer__207,knn_imputer__total_descent,imputer 100__205,cat_encoder__sport_cycling,cat_encoder__sport_fitness_equipment,cat_encoder__sport_generic,cat_encoder__sport_hiking,cat_encoder__sport_kayaking,cat_encoder__sport_rock_climbing,cat_encoder__sport_running,cat_encoder__sport_swimming,cat_encoder__sport_training,cat_encoder__sport_transition,cat_encoder__sport_walking,cycle_encoder__timestamp_sin,cycle_encoder__timestamp_cos,cycle_encoder__start_time_sin,cycle_encoder__start_time_cos
50,-0.37211,-0.585859,0.805755,-0.704545,0.0,-0.614973,-1.0,-0.31803,-0.290368,-0.75,-0.705292,-0.412607,-0.319958,0.780822,-0.340909,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.258819,-0.965926,0.258819,-0.965926


In [12]:
days = 30 
avg_activities_per_day = 2 
steps = days * avg_activities_per_day # sliding window size
# Prepare the training data
X_train = []
y_train = []

for i in tqdm(range(steps, activities.shape[0])):
# for i in range(steps, steps + 5):
    activity = activities.iloc[[i]]
    activity_time = activity["start_time"][i].strftime('%Y-%m-%d %H:%M:%S')
    window_df = garmin_data[garmin_data["beginTimestamp"] < activity_time].iloc[i - steps:i]
    X_train.append(preproc_garmin_data.transform(window_df))
    # TODO y_train.append(preproc_activity.transform(activity)[0]) 
    y_train.append(preproc_activity.transform(activity)) 

X_train, y_train = np.array(X_train), np.array(y_train)

print(f"{X_train.shape = } - {y_train.shape = }")

X_train.shape = (3303, 60, 66) - y_train.shape = (3303, 1, 31)


In [22]:
y_train = y_train.reshape((y_train.shape[0], y_train.shape[2])) # To remove, see above

In [23]:
y_train.shape

(3303, 31)

# Create model

In [26]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Activation
from keras.models import load_model

In [65]:
epochs = 100

model = Sequential()
model.add(LSTM(units=50, return_sequences = True, input_shape = (X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=y_train.shape[1]))
model.compile(loss="mse", optimizer="adam", metrics=['accuracy'])


In [66]:
model.fit(X_train, y_train, batch_size = 32, epochs = epochs)
model.summary

Epoch 1/100
104/104 [==============================] - 8s 33ms/step - loss: 1.0851 - accuracy: 0.1054
Epoch 2/100
104/104 [==============================] - 3s 27ms/step - loss: 1.0408 - accuracy: 0.1471
Epoch 3/100
104/104 [==============================] - 3s 29ms/step - loss: 1.0209 - accuracy: 0.1614
Epoch 4/100
104/104 [==============================] - 3s 28ms/step - loss: 1.0093 - accuracy: 0.1596
Epoch 5/100
104/104 [==============================] - 3s 29ms/step - loss: 1.0059 - accuracy: 0.1623
Epoch 6/100
104/104 [==============================] - 3s 27ms/step - loss: 0.9939 - accuracy: 0.1810
Epoch 7/100
104/104 [==============================] - 3s 28ms/step - loss: 0.9929 - accuracy: 0.1698
Epoch 8/100
104/104 [==============================] - 3s 27ms/step - loss: 0.9793 - accuracy: 0.1759
Epoch 9/100
104/104 [==============================] - 3s 25ms/step - loss: 0.9863 - accuracy: 0.1956
Epoch 10/100
104/104 [==============================] - 3s 26ms/step - loss: 0.974

<bound method Model.summary of <keras.engine.sequential.Sequential object at 0x7f33b771fd30>>

In [817]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 60, 50)            23400     
                                                                 
 dropout_4 (Dropout)         (None, 60, 50)            0         
                                                                 
 lstm_5 (LSTM)               (None, 60, 50)            20200     
                                                                 
 dropout_5 (Dropout)         (None, 60, 50)            0         
                                                                 
 lstm_6 (LSTM)               (None, 60, 50)            20200     
                                                                 
 dropout_6 (Dropout)         (None, 60, 50)            0         
                                                                 
 lstm_7 (LSTM)               (None, 50)               

In [29]:
garmin_data.shape

(3383, 52)

In [490]:
window_df = garmin_data.iloc[garmin_data.shape[0] - steps - 1:garmin_data.shape[0] - 1]
input_test = preproc_garmin_data.transform(window_df)

In [491]:
first_prediction = model.predict(np.array([input_test]))

1/1 [==============================] - 0s 51ms/step


In [492]:
first_prediction

array([[ 3.26048732e-01, -1.32820457e-01, -1.08376116e-01,
        -9.08865854e-02,  1.40819505e-01,  3.11233729e-01,
        -2.72869051e-01,  8.03724587e-01,  4.38185900e-01,
        -2.00406015e-02,  5.01094699e-01,  6.59387946e-01,
         3.36818665e-01, -1.06016859e-01,  6.52424574e-01,
        -2.16555190e+00,  3.65880966e-01, -1.04546745e-03,
        -9.07090453e-06,  2.26274505e-03,  1.68244320e-03,
        -1.03137444e-03,  1.87648207e-01,  1.68635771e-01,
         6.59881532e-03,  1.39871584e-02,  2.57512599e-01,
         3.26650262e-01, -5.19587398e-01,  4.69733834e-01,
        -3.37504923e-01]], dtype=float32)

In [493]:
# def initialize_model(input_shape):
#     model = Sequential()
#     model.add(layers.LSTM(64, input_shape=input_shape))
#     model.add(layers.Dense(input_shape[1], activation='softmax'))
#     model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#     return model

In [724]:
y_train[-1]
y_train.shape

(3303, 31)

In [495]:
model.evaluate(np.array([input_test]), np.array([y_train[-1]]))

1/1 [==============================] - 0s 53ms/step - loss: 0.5193 - accuracy: 0.0000e+00


[0.5192509293556213, 0.0]

# Inverse ?

In [754]:
from sklearn.compose import ColumnTransformer

class InvertableColumnTransformer(ColumnTransformer):
    """
    Adds an inverse transform method to the standard sklearn.compose.ColumnTransformer.
    """
    def inverse_transform(self, X):
        if isinstance(X,pd.DataFrame):
            X = X.to_numpy()

        transformer_to_cols = {}
        out = {}
        for t in self.transformers_:
            name = t[0]
            cols = t[-1]
            if name not in ("remainder", "cycle_encoder"):
                transformer_to_cols[name] = cols
                for col in cols:
                    out[col] = []
        for name, indices in self.output_indices_.items():
            transformer = self.named_transformers_.get(name, None)
            start = indices.start + 1 if indices.start != 0 else indices.start
            stop = indices.stop + 1 if indices.start != 0 else indices.stop
            
            arr = X[:, start -1 : stop - 1] if stop > X.shape[1] else X[:, start: stop] 

            if transformer in (None, "remainder", "passthrough", "drop"):
                pass

            else:
                try:
                    if isinstance(transformer, Pipeline):
                        tmp = transformer[1].inverse_transform(arr)
                        current_cols = transformer_to_cols[name]
                        for sub in tmp:
                            for col, val in zip(current_cols, sub):
                                out[col].append(val)
                    else:
                        # print(self.output_indices_)
                        transformer.inverse_transform(arr)
                except Exception as e:

                    print(e)

        return pd.DataFrame.from_dict(out)

In [796]:
def create_preproc_activity_2(activity_df):

    # get numerical data
    data_num = activity_df.select_dtypes(include=np.number)
    data_num

    # get cols numerical
    numerical_features = activity_df.select_dtypes(include=np.number).columns
    numerical_features = set(numerical_features) - set(["avg_stroke_distance",
                                                        "num_active_lengths",
                                                        "num_lengths",
                                                        "max_running_cadence",
                                                        "pool_length",
                                                        "max_cadence",
                                                        "avg_step_length",
                                                        "normalized_power",
                                                        "avg_power",
                                                        "total_strokes",
                                                        "training_stress_score",
                                                        "avg_cadence",
                                                        "max_power"
                                                        ])
    numerical_features = list(numerical_features)

    # Stamina features
    # 205: beginning
    # 206: ending
    # 207: min
    beginning_stamina_features = ["205"]
    numerical_features = list(
        set(numerical_features) - set(beginning_stamina_features))

    # pipeline knn imputer
    pipe_knn_imputer = Pipeline([
        ('knn_imputer', KNNImputer(n_neighbors=4)),
        ('robust_scaler', RobustScaler())
    ])

    # pipeline simple imputer fill value with 100 for beginning stamina (205)
    pipe_nan_to_100 = Pipeline([
        ('simple_imputer', SimpleImputer(strategy="constant", fill_value=100)),
        ('robust_scaler', RobustScaler())
    ])

    # get data and cols categorial feature
    data_cat = activity_df.select_dtypes(exclude=np.number)
    cat_features = data_cat.columns
    cat_features = list(set(cat_features) - set(["pool_length_unit"]))

    # pipeline categorical

    pipe_onehot = Pipeline([
        ("simple_imputer", SimpleImputer(strategy="most_frequent")),
        ("one_hot", OneHotEncoder(sparse_output=False,
         handle_unknown="ignore", drop="if_binary"))
    ])

    full_proces = InvertableColumnTransformer(transformers=[
        ("knn_imputer", pipe_knn_imputer, numerical_features),
        ("cat_encoder", pipe_onehot, ["sport"]),
        ("cycle_encoder", CyclicalFeaturesActivity(),
         ["timestamp", "start_time"]),
        ("imputer 100", pipe_nan_to_100, beginning_stamina_features)
    ]).set_output(transform="pandas")

    return full_proces

In [835]:
preproc = create_preproc_activity(activities)
preproc.fit(activities)
preproc

InvertableColumnTransformer(transformers=[('knn_imputer',
                                           Pipeline(steps=[('knn_imputer',
                                                            KNNImputer(n_neighbors=4)),
                                                           ('robust_scaler',
                                                            RobustScaler())]),
                                           ['training_load_peak',
                                            'avg_heart_rate', '206',
                                            'max_heart_rate',
                                            'total_anaerobic_training_effect',
                                            'total_calories', '188',
                                            'total_distance',
                                            'enhanced_avg_speed',
                                            'total_training_effect', '178',
                                            'total_ascent...
                                                            SimpleImputer(strategy='most_frequent')),
                                                           ('one_hot',
                                                            OneHotEncoder(drop='if_binary',
                                                                          handle_unknown='ignore',
                                                                          sparse_output=False))]),
                                           ['sport']),
                                          ('cycle_encoder',
                                           CyclicalFeaturesActivity(),
                                           ['timestamp', 'start_time']),
                                          ('imputer 100',
                                           Pipeline(steps=[('simple_imputer',
                                                            SimpleImputer(fill_value=100,
                                                                          strategy='constant')),
                                                           ('robust_scaler',
                                                            RobustScaler())]),
                                           ['205'])])

In [836]:
first_prediction

array([[ 3.26048732e-01, -1.32820457e-01, -1.08376116e-01,
        -9.08865854e-02,  1.40819505e-01,  3.11233729e-01,
        -2.72869051e-01,  8.03724587e-01,  4.38185900e-01,
        -2.00406015e-02,  5.01094699e-01,  6.59387946e-01,
         3.36818665e-01, -1.06016859e-01,  6.52424574e-01,
        -2.16555190e+00,  3.65880966e-01, -1.04546745e-03,
        -9.07090453e-06,  2.26274505e-03,  1.68244320e-03,
        -1.03137444e-03,  1.87648207e-01,  1.68635771e-01,
         6.59881532e-03,  1.39871584e-02,  2.57512599e-01,
         3.26650262e-01, -5.19587398e-01,  4.69733834e-01,
        -3.37504923e-01]], dtype=float32)

In [840]:
preproc.inverse_transform(first_prediction).iloc[0]

training_load_peak                    69.877296
avg_heart_rate                       117.425385
206                                   74.116966
max_heart_rate                       138.000992
total_anaerobic_training_effect         0.14082
total_calories                       725.802856
188                                    1.454262
total_distance                     21228.177734
enhanced_avg_speed                     4.383346
total_training_effect                  2.143886
178                                  764.599854
total_ascent                          93.531601
enhanced_max_speed                        6.337
207                                   73.815193
total_descent                         87.413361
sport                                   cycling
timestamp                               6:09.36
start_time                              5:14.14
205                                   99.662498
Name: 0, dtype: object

In [838]:
preproc.inverse_transform(np.array([y_train[-10]]))

,training_load_peak,avg_heart_rate,206,max_heart_rate,total_anaerobic_training_effect,total_calories,188,total_distance,enhanced_avg_speed,total_training_effect,178,total_ascent,enhanced_max_speed,207,total_descent,sport,timestamp,start_time,205
0,172.981277,147.0,44.0,164.0,0.0,888.0,3.0,15179.46,4.215,3.8,999.0,44.0,4.945,44.0,42.0,running,11:15.00,10:37.30,99.741181


In [764]:
_sin = 4.69733834e-01
_cos = -3.37504923e-01

In [515]:
np.arccos(_cos) / (2 * np.pi) * 24

7.314995547189589

In [516]:
np.arcsin(_sin) / (2 * np.pi) * 24

1.8678013675153158

In [765]:
import math
angle2 = math.atan2(_sin, _cos)  # ALWAYS USE THIS
angle2 *= 180 / math.pi
if angle2 < 0: angle2 += 360
print(angle2/24)

5.23738912903143


In [778]:
def convert_sin_cos_to_hour(s, c):
    angle = math.atan2(s, c)  # ALWAYS USE THIS
    angle *= 180 / math.pi
    if angle < 0: angle += 360
    print(angle / 24)

In [779]:

convert_sin_cos_to_hour(-1.00000000e+00, -1.83697020e-16 )

11.25


In [780]:
convert_sin_cos_to_hour(-9.65925826e-01, -2.58819045e-01)

10.625000000057799


In [772]:
   # 0.46973383 -0.33750492
convert_sin_cos_to_hour(0.32665026, -0.5195874)

6.160148252132713


In [773]:
convert_sin_cos_to_hour(0.46973383, -0.33750492)

5.2373891286091245


# Test From python code

In [ ]:
create_train_and_save_model()

✅ ../raw_data/Wellness/ loaded
✅ Wellness cleaned
✅ ../raw_data/Fitness/ loaded
✅ Fitness cleaned


/home/nico/code/nicblanc/overcome-tomorrow/overcome_tomorrow/utils/data.py:180: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_fitness_df["date"] = pd.to_datetime(clean_fitness_df["beginTimestamp"], unit="ms").dt.date
/home/nico/code/nicblanc/overcome-tomorrow/overcome_tomorrow/utils/data.py:181: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_fitness_df["date"] = pd.to_datetime(clean_fitness_df["date"])
/home/nico/code/nicblanc/overcome-tomorrow/overcome_tomorrow/utils/data.py:182: SettingWithCopy

✅ ../raw_data/Aggregator/ loaded
✅ Aggregator cleaned
✅ Datasets merged


 98%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍   | 3223/3303 [03:43<00:04, 17.31it/s]